In [1]:
import os

print(os.getcwd()) 
gt_stutter = 'libristutter_result\gt_libristutter.csv'
stutter_transcription = 'libristutter_result\wav2vec_libristutter.csv'

print(os.path.exists(gt_stutter))  # This will print True if folder1 exists, otherwise False
print(os.path.exists(stutter_transcription))  # This will print True if folder2 exists, otherwise False

c:\Users\xnishikawa\OneDrive - Olin College of Engineering\Documents\GitHub\stuttered-speech-asr
True
True


In [2]:
import pandas as pd

df_gt_stutter = pd.read_csv(gt_stutter)
df_stutter_transcription = pd.read_csv(stutter_transcription)

print(df_gt_stutter.shape)
print(df_stutter_transcription.shape)

(5601, 4)
(5590, 3)


In [3]:
# Remove the first column (index 0)
df_gt_stutter_clean = df_gt_stutter.drop(df_gt_stutter.columns[0], axis=1)

# Reset the index if needed
df_gt_stutter_clean = df_gt_stutter_clean.reset_index(drop=True)
df_gt_stutter_clean = df_gt_stutter_clean.drop('total_stutter', axis=1)
df_gt_stutter_clean['file_name'] = df_gt_stutter_clean['file_name'].str.replace('.csv', '', regex=False)

df_gt_stutter_clean.head

<bound method NDFrame.head of              file_name                                  gt_transcriptions
0     2893-139322-0043  stutter a swarthy hunchback ridiculously gay a...
1     2893-139322-0027  buy a stutter of coughing men each guiding a l...
2     2893-139322-0038  but for the most part stutter this remnant tha...
3     2893-139322-0011  stutter latter day labor male as well as femal...
4     2893-139322-0001  on their way the platform cross the thames stu...
...                ...                                                ...
5596   6000-55211-0028  then that it will bring in money and we need m...
5597   6000-55211-0005  that she was one day stutter to be mistress of...
5598   6000-55211-0022  nearest set a tree like plate of the same we'r...
5599   6000-55211-0034  brought of course if it's worth that stutter t...
5600   6000-55211-0031  or against uncle william for wanting the teapo...

[5601 rows x 2 columns]>

In [4]:
# Remove the first column (index 0)
df_stutter_transcription_clean = df_stutter_transcription.drop(df_stutter_transcription.columns[0], axis=1)

# Reset the index if needed
df_stutter_transcription_clean['file_name'] = df_stutter_transcription_clean['file_name'].str.replace('.flac', '', regex=False)

df_stutter_transcription_clean.head

<bound method NDFrame.head of             file_name                                         prediction
0     6925-80680-0006  the king made king made king made no remark bu...
1     6925-80680-0072  amer ali bowed low and answered that he had un...
2     6925-80680-0040  and as the storm had by this time passed over ...
3     6925-80680-0036  ameer ali thought that her voice sounded rathe...
4     6925-80680-0070  then the princess sent for her father and told...
...               ...                                                ...
5585  289-121652-0022  indeed as mister william h whitmore points out...
5586  289-121652-0007  are doubtless offshoots from ancient folklor s...
5587  289-121652-0004  are evidently drawn from a variety of sources ...
5588  289-121652-0011  although perreut's tals differ materially uly ...
5589  289-121652-0023  boston children at that date says mister whitm...

[5590 rows x 2 columns]>

In [5]:
import pandas as pd

# Merging dataframes based on the filename column
merged_df = pd.merge(df_gt_stutter_clean[['file_name', 'gt_transcriptions']], df_stutter_transcription_clean[['file_name', 'prediction']], on='file_name')

# Display the merged dataframe
print(merged_df.head)


<bound method NDFrame.head of              file_name                                  gt_transcriptions  \
0     2893-139322-0043  stutter a swarthy hunchback ridiculously gay a...   
1     2893-139322-0027  buy a stutter of coughing men each guiding a l...   
2     2893-139322-0038  but for the most part stutter this remnant tha...   
3     2893-139322-0011  stutter latter day labor male as well as femal...   
4     2893-139322-0001  on their way the platform cross the thames stu...   
...                ...                                                ...   
5528   6000-55211-0028  then that it will bring in money and we need m...   
5529   6000-55211-0005  that she was one day stutter to be mistress of...   
5530   6000-55211-0022  nearest set a tree like plate of the same we'r...   
5531   6000-55211-0034  brought of course if it's worth that stutter t...   
5532   6000-55211-0031  or against uncle william for wanting the teapo...   

                                             

In [6]:
from sklearn.model_selection import train_test_split

# Splitting the merged dataframe into training and testing sets
train_df, test_df = train_test_split(merged_df, test_size=0.2, random_state=42)

# Exporting the training and testing sets to separate CSV files
train_df.to_csv('train_data.csv', index=False)
test_df.to_csv('test_data.csv', index=False)

merged_df.to_csv('merged_data.csv', index=False)

In [7]:
# Validation that the data was correct
%pip install jiwer




Note: you may need to restart the kernel to use updated packages.


In [10]:
import jiwer

# Calculate WER for each pair of ground truth and prediction
ground_truth_list = merged_df['gt_transcriptions'].tolist()
prediction_list = merged_df['prediction'].tolist()

# Calculate WER for each row
wer_results = [
    jiwer.wer(gt, pred)
    for gt, pred in zip(ground_truth_list, prediction_list)
]

# Add WER results to the dataframe
merged_df['wer'] = wer_results

# Calculate the mean WER
mean_wer = merged_df['wer'].mean()

print(f"The mean Word Error Rate is: {mean_wer:.4f}")


The mean Word Error Rate is: 0.4254
